Considera el problema de la planificación semanal de la producción de una empresa de fabricación de automóviles. La empresa fabrica $M$ modelos de automóvil. La producción de estos modelos requiere la ejecución de $T$ tareas (por ejemplo, ensamblaje de componentes mecánicas, pintura, perfilado, montaje de componentes eléctricas, etc.). La realización de estas tareas requiere personal cualificado, en un número de operarios que es diferente para cada modelo de automóvil. Sea $o_{mt}, m = 1, \ldots, M; t = 1, \ldots, T$, el número de operarios especializados en la tarea $t$ que se necesita para fabricar un automóvil modelo $m$ a la semana. Sabiendo que $\overline{o}_{t}$, con $m = 1, \ldots, M$ y $t = 1, \ldots, T$ representa el número máximo de operarios cualificados para ejecutar la tarea $t$ de los que la empresa dispone en una semana laboral y que el beneficio que la empresa obtiene por la venta del modelo $m$ de automóvil es $b_m$, se pide:

1.   Construye un modelo de programación entera en AMPL para maximizar el beneficio semanal que hace la empresa a partir de la fabricación de los diferentes modelos de automóviles (4 puntos).

In [ ]:
%pip install -q amplpy

In [ ]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Pe = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [ ]:
Pe.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0, integer;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];

""")


Considera el caso particular en que la empresa fabrica tres modelos de automóviles ($M = 3$) cuyo montaje requiere de hasta cuatro tareas especializadas ($T = 4$). Los operarios disponibles para la ejecución de cada tarea, el número de ellos que requiere cada modelo de automóvil y los beneficios (en miles de €) que obtiene la empresa de la venta de cada modelo, son los especificados en las siguientes tablas.

<center>

|  Modelo    | Tarea 1   | Tarea 2  | Tarea 3 | Tarea 4 | Beneficio|
| ------------  | --------     | ---      | ---    |  ---    |--- |
| 1            | 1       | 1 | 1|  1 |  12
| 2            | 2       | 2 | 1| 3  |  20
| 3            | 1       | 2 | 3|  0  |  9
| **#Operarios**   | 7   | 10      | 8   |  6    |

</center>



*   Resuelve este caso particular con un solver para programas lineales enteros (1 punto).

In [ ]:
# Introducimos el beneficio, el número de operarios para cada tarea que requiere
# cada modelo de automóvil y el número máximo de operarios especializados en
# la ejecución de cada tarea.

import pandas as pd
import numpy as np

beneficio ={
    1: 12,
    2: 20,
    3: 9,
}

max_operarios = {
    1: 7,
    2: 10,
    3: 8,
    4: 6,
}

 # Creamos un data frame que contenga el número de operarios necesarios por
 # tarea y modelo

Operarios_por_tarea_modelo = pd.DataFrame(
        np.array(
            [
                [1, 1, 1, 1],
                [2, 2, 1, 3],
                [1, 2, 3, 0]
            ]
        ),
        columns= list(max_operarios.keys()),
        index  = list(beneficio.keys()),
    )

Pe.param["M"] = len(beneficio)
Pe.param["T"] = len(max_operarios)
Pe.param["b"] = {m: b for m, b in beneficio.items()}
Pe.param["omax"] = {t: o for t, o in max_operarios.items()}
Pe.get_parameter("o").set_values(Operarios_por_tarea_modelo)



In [ ]:
Pe.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 72
2 simplex iterations


In [ ]:
x = Pe.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = Pe.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


   x.val
1      6
2      0
3      0
Beneficio: 72.0


Resuelve este caso particular mediante el *algoritmo de ramificación y acotación*, utilizando un solver adecuado para resolver los problemas lineales relajados que sean necesarios. Representa el árbol al que la aplicación del algoritmo ha dado lugar, especificando los tipos de podas que has realizado en cada caso hasta certificar la solución óptima del problema entero (5 puntos).

Problema relajado Pr:

In [ ]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Pr = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [ ]:
Pr.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];

""")


In [ ]:
Pr.param["M"] = len(beneficio)
Pr.param["T"] = len(max_operarios)
Pr.param["b"] = {m: b for m, b in beneficio.items()}
Pr.param["omax"] = {t: o for t, o in max_operarios.items()}
Pr.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
Pr.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 78
3 simplex iterations


In [ ]:
x = Pr.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = Pr.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


      x.val
1  6.000000
2  0.000000
3  0.666667
Beneficio: 78.0


Ramificamos $x_3$

In [ ]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

P1 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [ ]:
P1.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];
    bounding: x[3] <= 0;

""")


In [ ]:
P1.param["M"] = len(beneficio)
P1.param["T"] = len(max_operarios)
P1.param["b"] = {m: b for m, b in beneficio.items()}
P1.param["omax"] = {t: o for t, o in max_operarios.items()}
P1.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
P1.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 72
1 simplex iterations


In [ ]:
x = P1.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = P1.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


   x.val
1      6
2      0
3      0
Beneficio: 72.0


In [ ]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

P2 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [ ]:
P2.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];
    bounding: x[3] >= 1;

""")


In [ ]:
P2.param["M"] = len(beneficio)
P2.param["T"] = len(max_operarios)
P2.param["b"] = {m: b for m, b in beneficio.items()}
P2.param["omax"] = {t: o for t, o in max_operarios.items()}
P2.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
P2.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 73
3 simplex iterations


In [ ]:
x = P2.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = P2.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


   x.val
1    4.5
2    0.5
3    1.0
Beneficio: 73.0


A partir del problema $P_2$, ramificamos $x_2$

In [ ]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

P3 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [ ]:
P3.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];
    bounding1: x[3] >= 1;
    bounding2: x[2] <= 0;

""")


In [ ]:
P3.param["M"] = len(beneficio)
P3.param["T"] = len(max_operarios)
P3.param["b"] = {m: b for m, b in beneficio.items()}
P3.param["omax"] = {t: o for t, o in max_operarios.items()}
P3.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
P3.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 69
2 simplex iterations


In [ ]:
x = P3.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = P3.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


   x.val
1      5
2      0
3      1
Beneficio: 69.0


In [ ]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

P4 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [ ]:
P4.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];
    bounding1: x[3] >= 1;
    bounding2: x[2] >= 1;

""")


In [ ]:
P4.param["M"] = len(beneficio)
P4.param["T"] = len(max_operarios)
P4.param["b"] = {m: b for m, b in beneficio.items()}
P4.param["omax"] = {t: o for t, o in max_operarios.items()}
P4.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
P4.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 68
3 simplex iterations


In [ ]:
x = P4.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = P4.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


      x.val
1  3.000000
2  1.000000
3  1.333333
Beneficio: 68.0


El árbol queda agotado y la solución óptima es la proporcionada por P1.

Veamos qué pasa si, a partir de $P_2$, hubiéramos ramificado $x_1$ en lugar de $x_2$.

In [ ]:
P3.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];
    bounding1: x[3] >= 1;
    bounding2: x[1] <= 4;

""")


In [ ]:
P3.param["M"] = len(beneficio)
P3.param["T"] = len(max_operarios)
P3.param["b"] = {m: b for m, b in beneficio.items()}
P3.param["omax"] = {t: o for t, o in max_operarios.items()}
P3.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
P3.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 71.33333333
3 simplex iterations


In [ ]:
x = P3.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = P3.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


      x.val
1  4.000000
2  0.666667
3  1.111111
Beneficio: 71.33333333333333


Esta rama se poda por acotación.

In [ ]:
P4.eval(r"""
reset;
# Parámetros
param M >= 0;  # Número de modelos de automóviles
param T >= 0;  # Número de tareas
param o {1..M, 1..T} >= 0;  # Número de operarios necesarios para cada tarea
param omax {1..T} >= 0;  # Número máximo de operarios disponibles para cada tarea
param b {1..M};  # Beneficio por la venta de cada modelo de automóvil

# Variables de decisión
var x {1..M} >= 0;  # Cantidad de unidades del modelo m a producir

# Función objetivo (maximizar el beneficio)
maximize Beneficio: sum {m in 1..M} b[m] * x[m];
subject to Operarios_por_tarea {t in 1..T}:
    sum {m in 1..M} o[m, t] * x[m] <= omax[t];
    bounding1: x[3] >= 1;
    bounding2: x[1] >= 5;

""")


In [ ]:
P4.param["M"] = len(beneficio)
P4.param["T"] = len(max_operarios)
P4.param["b"] = {m: b for m, b in beneficio.items()}
P4.param["omax"] = {t: o for t, o in max_operarios.items()}
P4.get_parameter("o").set_values(Operarios_por_tarea_modelo)

In [ ]:
P4.solve(solver="cplex") # Resolvemos con el solver "cplex"


Solution determined by presolve;
objective Beneficio = 69.


In [ ]:
x = P4.get_variable("x") # Producción de modelos de automóvil
df_x = x.get_values().to_pandas()
profit = P4.get_objective("Beneficio").value()
print(df_x)
print(f"Beneficio: {profit}")


   x.val
1      5
2      0
3      1
Beneficio: 69.0


Esta rama se poda por acotación también y por tanto, llegamos de nuevo a la conclusión de que la solución del problema entero es la proporcionada por el problema $P_1$.